Initialize the local model

In [19]:
from langchain_community.llms import Ollama 

llm = Ollama(model = "mistral")

In [20]:
llm.invoke("tell me  a short joke")

" Why don't scientists trust atoms?\n\nBecause they make up everything!"

Load data for RAG

In [21]:
from langchain_community.document_loaders import WebBaseLoader 

loader = WebBaseLoader(
    web_path="https://blog.langchain.dev/langgraph/"
)

docs = loader.load()

Index the data

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter 

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True 
)



In [23]:
all_splits = text_splitter.split_documents(docs)

In [24]:
from langchain_community import embeddings 

embedding = embeddings.ollama.OllamaEmbeddings(
    model="nomic-embed-text"
)

In [25]:
from langchain_community.vectorstores import Chroma 

vectorstore = Chroma.from_documents(
    documents = all_splits,
    embedding = embedding
)

In [26]:
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":6}
)

In [27]:
retriever.get_relevant_documents("What is LangGraph?")

[Document(page_content='"model",\n    should_continue,\n    {\n        "end": END,\n        "continue": "tools"\n    }\n)CompileAfter we define our graph, we can compile it into a runnable! This simply takes the graph definition we\'ve created so far an returns a runnable. This runnable exposes all the same method as LangChain runnables (.invoke, .stream, .astream_log, etc) allowing it to be called in the same manner as a chain.app = graph.compile()Agent ExecutorWe\'ve recreated the canonical LangChain AgentExecutor with LangGraph. This will allow you to use existing LangChain agents, but allow you to more easily modify the internals of the AgentExecutor. The state of this graph by default contains concepts that should be familiar to you if you\'ve used LangChain agents: input, chat_history, intermediate_steps (and agent_outcome to represent the most recent agent outcome)from typing import TypedDict, Annotated, List, Union\nfrom langchain_core.agents import AgentAction, AgentFinish', m

Create a chain to contextualize

In [28]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder 

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

In [29]:
contextualize_q_prompt= ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [30]:
from langchain_core.output_parsers import StrOutputParser

contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

In [31]:
from langchain_core.messages import AIMessage, HumanMessage

contextualize_q_chain.invoke(
    {
        "chat_history":[
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model"),
        ],
        "question": "What is meant by large?",
    }
)

' What does "large" refer to in the context of a "large language model"?'

Create a chain for chat history

In [32]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

In [33]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [34]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [35]:
def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]

In [36]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    RunnablePassthrough.assign(
        context = contextualized_question | retriever | format_docs
    )
    | qa_prompt 
    | llm
)

Inference

In [37]:
chat_history = []

question = "What is LangGraph?"
ai_msg = rag_chain.invoke(
    {
        "question": question,
        "chat_history": chat_history
    }
)

In [38]:
ai_msg

" LangGraph is a modification of LangChain's AgentExecutor that exposes its logic in a more natural and modifiable way. It allows users to make modifications such as forcing an agent to call a tool first, adding a human-in-the-loop step before calling tools, managing agent steps for custom handling of intermediate steps, and making the agent return output in a specific format or directly from a tool (only for Chat Agent Executor)."

In [39]:
chat_history.extend(
    [
        HumanMessage(content=question), ai_msg
    ]
)

In [40]:
second_question = "What is it used for?"

rag_chain.invoke(
    {
        "question": second_question,
        "chat_history": chat_history
    }
)

" LangGraph is a modification of LangChain's AgentExecutor that provides a more natural and modifiable way to use LangChain agents. It allows users to customize agent behavior in various ways, such as calling tools first, adding human-in-the-loop steps, managing intermediate steps, and controlling tool output format or direct return (for Chat Agent Executor)."